In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [9]:
# Gather Data
boston_dataset = load_boston()

data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)

features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])


In [12]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

property_stats = np.ndarray(shape=(1,11))

property_stats[0][CRIME_IDX] = features['CRIM'].mean()
property_stats[0][ZN_IDX ] = features['ZN'].mean()
property_stats[0][CHAS_IDX] = features['CHAS'].mean()

property_stats


array([[3.61352356e+000, 1.13636364e+001, 6.91699605e-002,
        6.91225332e-310, 6.91225332e-310, 6.91225332e-310,
        6.91225332e-310, 6.91225332e-310, 6.91225332e-310,
        6.91230651e-310, 6.91225332e-310]])

In [16]:
features.mean()
features.mean().values
features.mean().values.reshape(1,11)
property_stats = features.mean().values.reshape(1,11)
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [41]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [44]:
def get_log_estimate(nr_rooms,
                    students_per_classroom,
                    next_to_river = False,
                    high_confidence= True):
    
    # Configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
            
    # Make Prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calc range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
        
    return log_estimate, upper_bound, lower_bound, interval

In [47]:
get_log_estimate(5, 10,next_to_river = True, high_confidence=False)

(3.3323627931260953, 3.5198749283232256, 3.144850657928965, 68)

In [48]:
np.median(boston_dataset.target)


21.2

In [55]:
ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(9, students_per_classroom=15,
                                               next_to_river = False, high_confidence=False)

# Convert to today's dollars
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_high = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

rounded_est = np.around(dollar_est, -3)
rounded_high = np.around(dollar_high, -3)
rounded_low = np.around(dollar_low, -3)

print(f'The estimated property values is {rounded_est}')
print(f'at {conf}% confidence the valuation range is')
print(f'USD {rounded_low} at the lower end to USD {rounded_high} at the higher end.')

The estimated property values is 827000.0
at 68% confidence the valuation range is
USD 685000.0 at the lower end to USD 997000.0 at the higher end.


In [59]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True ):
    
    """
    Estimate the price of a property in boston
    
    Keyword arguments:
    rm -- number of rooms in the property
    ptratio -- number of students per teacher in the classroom for the school in the area
    chas -- True if the property is next to the river, False otherwise
    large_range -- True for a 95% prediction interval, False for a 68% interval
    """
    
    if rm < 1 or ptratio < 1:
        print('Houses at least one rooms to exists, or there needs to be at least one student')
        return
    
    
    log_est, upper, lower, conf = get_log_estimate(nr_rooms=rm, students_per_classroom=ptratio,
                                               next_to_river = chas, high_confidence=large_range)

    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_high = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    rounded_est = np.around(dollar_est, -3)
    rounded_high = np.around(dollar_high, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated property values is {rounded_est}')
    print(f'at {conf}% confidence the valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_high} at the higher end.')

In [61]:
get_dollar_estimate(rm=2, ptratio=30, chas=True)

The estimated property values is 278000.0
at 95% confidence the valuation range is
USD 191000.0 at the lower end to USD 404000.0 at the higher end.


In [1]:
import boston_valuation as val

In [2]:
val.get_dollar_estimate(6,12,True)

The estimated property values is 783000.0
at 95% confidence the valuation range is
USD 538000.0 at the lower end to USD 1139000.0 at the higher end.
